In [ ]:
# region 1. Setup & Training Function
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split, Subset, WeightedRandomSampler
from torchvision import datasets, models, transforms
from torchvision.models import EfficientNet_V2_S_Weights
from torch.utils.checkpoint import checkpoint
import matplotlib.pyplot as plt
import time
import copy
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import sys
import warnings
import os
from PIL import Image

warnings.filterwarnings("ignore")

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
optimizations = []

if DEVICE.type == 'cuda':
    if torch.cuda.is_bf16_supported():
        AMP_DTYPE = torch.bfloat16
        optimizations.append("BFloat16")
    else:
        raise RuntimeError("❌ GPU tidak support BF16!")
        
    if torch.backends.cuda.matmul.allow_tf32:
        optimizations.append("TF32")
    if torch.backends.cudnn.benchmark:
        optimizations.append("cuDNN Benchmark")
    optimizations.append("Channels Last")

print(f"=== SYSTEM INFO ===")
print(f"Device     : {torch.cuda.get_device_name(0)}")
print(f"Optimasi   : {', '.join(optimizations)}")
print("===================")

class CheckpointWrapper(nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module
    def forward(self, x):
        return checkpoint(self.module, x, use_reentrant=False)

class TransformedSubset(torch.utils.data.Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform: x = self.transform(x)
        return x, y
    def __len__(self):
        return len(self.subset)

def train_model(model, criterion, optimizer, scheduler, dataloaders, device, num_epochs, dataset_sizes, phase_name="Training"): 
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}
    
    print(f"\n--- Memulai {phase_name} ({num_epochs} Epochs) ---")

    for epoch in range(num_epochs):
        current_lr = optimizer.param_groups[0]['lr']
        history['lr'].append(current_lr)
        
        print(f"\nEpoch {epoch+1}/{num_epochs} [LR: {current_lr:.2e}]")
        
        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss, running_corrects = 0.0, 0
            
            pbar = tqdm(dataloaders[phase], desc=f"  {phase.upper()}", leave=False)

            for inputs, labels in pbar:
                inputs = inputs.to(device, memory_format=torch.channels_last, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                optimizer.zero_grad(set_to_none=True)
                
                with torch.set_grad_enabled(phase == 'train'), \
                     torch.autocast(device_type=device.type, dtype=AMP_DTYPE, enabled=True): 
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{torch.sum(preds == labels.data)/inputs.size(0):.4f}'})

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print(f"  {phase.upper()} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")

            if phase == 'train':
                history['train_loss'].append(epoch_loss)
                history['train_acc'].append(epoch_acc.item())
            else:
                history['val_loss'].append(epoch_loss)
                history['val_acc'].append(epoch_acc.item())
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    try:
                        torch.save(model.state_dict(), f'best_model_{phase_name.replace(" ", "_").lower()}_temp.pth')
                    except: pass
        
        if scheduler: scheduler.step()

    time_elapsed = time.time() - since
    print(f"\n{phase_name} Selesai: {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s | Best Acc: {best_acc:.4f}")
    try: model.load_state_dict(best_model_wts)
    except: pass
    return model, history
# endregion

In [ ]:
# region 2. Config & Data Loading
DATASET_PATH = 'Dataset'
IMG_SIZE = 224

BATCH_SIZE_EXTRACT = 192
BATCH_SIZE_TUNE = 12
    
EPOCHS_FEATURE_EXTRACT = 20
EPOCHS_FINE_TUNE = 30
    
VALIDATION_SPLIT = 0.2 
LR_FEATURE_EXTRACT = 1e-2 
LR_FINE_TUNE = 1e-3
WEIGHT_DECAY = 1e-4
LABEL_SMOOTHING = 0.1
    
PLOT_FILENAME = 'training_results_final.png'
CONFUSION_MATRIX_FILENAME = 'confusion_matrix_final.png'
BEST_MODEL_EXTRACT_PATH = 'best_model_extract.pth'
MODEL_SAVE_PATH = 'citrus_efficientnetv2s_final.pth'

class InMemoryDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.samples = []
        
        temp_dataset = datasets.ImageFolder(root_dir)
        self.classes = temp_dataset.classes
        self.targets = temp_dataset.targets
        
        print(f"(!) Memuat {len(temp_dataset)} gambar ke RAM...")
        for path, target in tqdm(temp_dataset.samples, desc="Loading to RAM"):
            with open(path, 'rb') as f:
                img = Image.open(f).convert('RGB')
                img = img.resize((IMG_SIZE, IMG_SIZE))
                self.samples.append((img, target))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        img, target = self.samples[index]
        if self.transform:
            img = self.transform(img)
        return img, target

def prepare_data():
    weights = EfficientNet_V2_S_Weights.DEFAULT
    preprocess = weights.transforms(antialias=True)

    train_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        preprocess
    ])
    val_transforms = preprocess

    try:
        full_dataset = InMemoryDataset(DATASET_PATH, transform=None)
        
        CLASSES = sorted(full_dataset.classes)
        NUM_CLASSES = len(CLASSES)
        
        print("=== DATASET INFO ===")
        print(f"Total Gambar : {len(full_dataset)}")
        print(f"Model        : EfficientNetV2-S")
        print(f"Batch Size   : FE={BATCH_SIZE_EXTRACT}, FT={BATCH_SIZE_TUNE}")
        print("====================")

        targets = [s[1] for s in full_dataset.samples]
        class_counts = np.bincount(targets)
        class_weights = [len(full_dataset) / c for c in class_counts]
        
        train_size = int((1 - VALIDATION_SPLIT) * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_indices, val_indices = random_split(range(len(full_dataset)), [train_size, val_size], generator=torch.Generator().manual_seed(42))

        train_targets = [targets[i] for i in train_indices]
        sample_weights = [class_weights[t] for t in train_targets]
        sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(train_indices), replacement=True)

        train_dataset = TransformedSubset(Subset(full_dataset, train_indices), train_transforms)
        val_dataset = TransformedSubset(Subset(full_dataset, val_indices), val_transforms)

        num_workers = 0 
        
        dataloaders_extract = {
            'train': DataLoader(train_dataset, batch_size=BATCH_SIZE_EXTRACT, sampler=sampler, num_workers=num_workers, pin_memory=True),
            'val': DataLoader(val_dataset, batch_size=BATCH_SIZE_EXTRACT, shuffle=False, num_workers=num_workers, pin_memory=True)
        }
        dataloaders_tune = {
            'train': DataLoader(train_dataset, batch_size=BATCH_SIZE_TUNE, sampler=sampler, num_workers=num_workers, pin_memory=True),
            'val': DataLoader(val_dataset, batch_size=BATCH_SIZE_TUNE, shuffle=False, num_workers=num_workers, pin_memory=True)
        }
        dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
        
        return dataloaders_extract, dataloaders_tune, dataset_sizes, CLASSES, NUM_CLASSES, weights

    except Exception as e:
        print(f"Error Dataset: {e}")
        sys.exit(1)

if __name__ == '__main__':
    dataloaders_extract, dataloaders_tune, dataset_sizes, class_names, num_classes, weights = prepare_data()
# endregion

In [ ]:
# region 3. Feature Extraction
if __name__ == '__main__':
    print("\n=== TAHAP 1: FEATURE EXTRACTION ===")
    
    if 'weights' not in locals():
         dataloaders_extract, dataloaders_tune, dataset_sizes, class_names, num_classes, weights = prepare_data()

    model_extract = models.efficientnet_v2_s(weights=weights)

    for i in range(len(model_extract.features)):
        model_extract.features[i] = CheckpointWrapper(model_extract.features[i])
    for param in model_extract.features.parameters():
        param.requires_grad = False

    model_extract.classifier = nn.Sequential(
        nn.Dropout(p=0.4, inplace=True),
        nn.Linear(model_extract.classifier[1].in_features, num_classes)
    )
    
    model_extract = model_extract.to(DEVICE, memory_format=torch.channels_last)

    criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)
    
    optimizer = optim.SGD(model_extract.classifier.parameters(), lr=LR_FEATURE_EXTRACT, momentum=0.9, weight_decay=WEIGHT_DECAY)
    
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS_FEATURE_EXTRACT)

    model_extract, history_extract = train_model(
        model_extract, criterion, optimizer, scheduler, dataloaders_extract, DEVICE, 
        EPOCHS_FEATURE_EXTRACT, dataset_sizes, phase_name="Feature Extraction"
    )
    
    torch.save(model_extract.state_dict(), BEST_MODEL_EXTRACT_PATH)
    print(f"Checkpoint tersimpan: {BEST_MODEL_EXTRACT_PATH}")
# endregion

In [ ]:
# region 4. Fine Tuning
if __name__ == '__main__':
    print("\n=== TAHAP 2: FINE-TUNING ===")
    
    if 'weights' not in locals():
         dataloaders_extract, dataloaders_tune, dataset_sizes, class_names, num_classes, weights = prepare_data()
         
    model_tune = models.efficientnet_v2_s(weights=None) 

    num_ftrs_tune = model_tune.classifier[1].in_features
    model_tune.classifier = nn.Sequential(
        nn.Dropout(p=0.4, inplace=True),
        nn.Linear(num_ftrs_tune, num_classes)
    )

    for i in range(len(model_tune.features)):
        model_tune.features[i] = CheckpointWrapper(model_tune.features[i])

    try:
        model_tune.load_state_dict(torch.load(BEST_MODEL_EXTRACT_PATH, map_location='cpu'))
        print("Bobot FE dimuat.")
    except Exception as e:
        print(f"Load error: {e}, pakai init awal.")
        model_tune.load_state_dict(model_extract.state_dict())

    for param in model_tune.parameters():
        param.requires_grad = True

    model_tune = model_tune.to(DEVICE, memory_format=torch.channels_last)

    optimizer_tune = optim.SGD(model_tune.parameters(), lr=LR_FINE_TUNE, momentum=0.9, weight_decay=WEIGHT_DECAY)
    
    scheduler_tune = optim.lr_scheduler.CosineAnnealingLR(optimizer_tune, T_max=EPOCHS_FINE_TUNE)

    model_final, history_tune = train_model(
        model_tune, criterion, optimizer_tune, scheduler_tune, dataloaders_tune, DEVICE, 
        EPOCHS_FINE_TUNE, dataset_sizes, phase_name="Fine-Tuning"
    )

    torch.save(model_final.state_dict(), MODEL_SAVE_PATH)
    print(f"Model Final tersimpan: {MODEL_SAVE_PATH}")
# endregion

In [ ]:
# region 5. Visualization & Evaluation
from sklearn.metrics import classification_report

if __name__ == '__main__':
    print(f"\n=== VISUALISASI & EVALUASI ===")
    
    combined_history = {
        'train_acc': history_extract.get('train_acc', []) + history_tune.get('train_acc', []),
        'val_acc': history_extract.get('val_acc', []) + history_tune.get('val_acc', []),
        'train_loss': history_extract.get('train_loss', []) + history_tune.get('train_loss', []),
        'val_loss': history_extract.get('val_loss', []) + history_tune.get('val_loss', [])
    }

    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(combined_history['train_acc'], label='Train Acc')
    plt.plot(combined_history['val_acc'], label='Val Acc', marker='o')
    plt.title('Accuracy History')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(combined_history['train_loss'], label='Train Loss')
    plt.plot(combined_history['val_loss'], label='Val Loss', marker='o')
    plt.title('Loss History')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(PLOT_FILENAME)
    plt.show()

    print(f"\nMenghitung Metrik Lengkap...")
    if 'model_final' in locals():
        model_final.eval()
        all_preds, all_labels = [], []
        
        with torch.no_grad():
            for inputs, labels in tqdm(dataloaders_tune['val'], desc="Evaluasi Final"):
                inputs = inputs.to(DEVICE, memory_format=torch.channels_last)
                labels = labels.to(DEVICE)
                
                with torch.autocast(device_type=DEVICE.type, dtype=torch.bfloat16, enabled=True):
                    outputs = model_final(inputs)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        print("\n" + "="*60)
        print("LAPORAN KLASIFIKASI DETAIL")
        print("="*60)
        print(classification_report(all_labels, all_preds, target_names=class_names, digits=4))
        
        cf_matrix = confusion_matrix(all_labels, all_preds)
        
        row_sums = cf_matrix.sum(axis=1)[:, np.newaxis]
        row_sums[row_sums == 0] = 1 
        percentages = cf_matrix.astype('float') / row_sums
        
        annot_labels = np.asarray([
            [f"{count}\n({pct:.1%})" for count, pct in zip(row, pct_row)]
            for row, pct_row in zip(cf_matrix, percentages)
        ])

        plt.figure(figsize=(12, 10))
        sns.heatmap(cf_matrix, annot=annot_labels, fmt='', cmap='Blues', 
                    xticklabels=class_names, yticklabels=class_names,
                    annot_kws={"size": 10})
        
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.title('Confusion Matrix (Count & Percentage)')
        plt.tight_layout()
        plt.savefig(CONFUSION_MATRIX_FILENAME)
        plt.show()
    else:
        print("Model final tidak ditemukan.")

    print("\n--- PROSES SELESAI ---")
# endregion